In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
from utils import load_img, resize_crop, imshow, interleave
%matplotlib inline

In [2]:
img_directory = '/home/ec2-user/img_align_celeba'
batch_size = 64
training_set_size = 2048
img_size = 64

In [3]:
training = np.array([resize_crop(load_img(i+1, img_directory), (img_size, img_size)) for i in range(training_set_size)])

/home/ec2-user/.conda/envs/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
from autoencoder import Autoencoder
from discriminator import Discriminator
from keras.optimizers import Adam

vae = Autoencoder(img_shape=(img_size, img_size, 3), batch_size=batch_size)
vae.build_model()

disc = Discriminator(img_shape=(img_size, img_size, 3), batch_size=batch_size)
disc.build_model()

Using TensorFlow backend.


In [5]:
optimizer = Adam(lr=.00001)

In [6]:
def generate_disc_training_data(images, vae, batch_size, img_rows, img_cols, channels=3):
    data_set_size = images.shape[0]
    # real images encoded, then decoded
    encdec = vae.model.predict(images, batch_size=batch_size)
    # sampled from z then decoded. Do for as many as number of images.
    sampled = vae.decoder.predict(np.random.normal(size=(data_set_size, vae.zsize)), batch_size=batch_size)
        
    training = interleave(images, encdec, sampled)
    # 0 for real, 1 for fake
    labels = interleave(np.zeros(data_set_size), np.ones(data_set_size), np.ones(data_set_size))
    return (training, labels)

import keras.backend as K
def decoder_loss_factory(disc):
    def decoder_loss(y_true, y_pred):
        #similarity = disc.diff_loss(y_true, y_pred)
        discriminator_out = K.log(1 - disc.model(y_pred))
#         return similarity - discriminator_out
        return discriminator_out
    return decoder_loss

# Debugging

In [7]:
disc_training, disc_labels = generate_disc_training_data(training, vae, batch_size, img_size, img_size)

In [8]:
# train disc
# disc.model.compile(optimizer, 'binary_crossentropy')
# disc.model.fit(disc_training, disc_labels, epochs=20, batch_size=batch_size, verbose=True)

In [27]:
vae.decoder.layers

In [28]:
len(vae.decoder.layers)

20

In [31]:
vae.decoder.layers[19].output_shape

(None, 64, 64, 3)

In [37]:
vae.decoder.output

AttributeError: Layer model_2 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [9]:
# train whole vae
vae.model.compile(optimizer, disc.diff_loss)
vae.model.fit(training, training, epochs=1, batch_size=batch_size, verbose=True)

 

(None, None, None, None)


ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 256). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.